# Bước 4: FP-Growth Modeling for Association Rules

Notebook này sử dụng ma trận `basket_bool` (được chuẩn bị ở Bước 02) để:

- Khai thác tập mục phổ biến (frequent itemsets) bằng thuật toán FP-Growth
- Sinh luật kết hợp (association rules) với các chỉ số: `support`, `confidence`, `lift`
- Lọc luật theo các ngưỡng do người dùng cấu hình
- Trực quan hoá một số nhóm luật tiêu biểu phục vụ storytelling & phân tích kinh doanh
- So sánh sơ bộ thời gian chạy và số lượng luật thu được so với Apriori (sẽ chi tiết hơn ở Bước 5)

Notebook được thiết kế theo kiểu parameterized để dễ dàng tích hợp với papermill.


In [ ]:
# PARAMETERS (for papermill)

# Đường dẫn tới basket_bool được tạo từ Notebook 02
BASKET_BOOL_PATH = "data/processed/basket_bool.parquet"

# Đường dẫn lưu file luật kết hợp sau khi lọc (FP-Growth)
RULES_OUTPUT_PATH = "data/processed/rules_fpgrowth_filtered.csv"

# Tham số cho bước khai thác tập mục phổ biến (frequent itemsets)
MIN_SUPPORT = 0.03  # ngưỡng support tối thiểu
MAX_LEN = 4         # độ dài tối đa của itemset (số sản phẩm trong 1 tập)

# Tham số cho bước sinh luật
METRIC = "lift"     # chỉ số dùng để generate rules: 'support' / 'confidence' / 'lift'
MIN_THRESHOLD = 1.0 # ngưỡng tối thiểu cho METRIC

# Tham số lọc luật sau khi generate
FILTER_MIN_SUPPORT = 0.01
FILTER_MIN_CONF = 0.3
FILTER_MIN_LIFT = 1.2
FILTER_MAX_ANTECEDENTS = 2
FILTER_MAX_CONSEQUENTS = 1

# Số lượng luật top để vẽ biểu đồ
TOP_N_RULES = 20

# Thêm các tham số này vào phần PARAMETERS ở đầu notebook:

# Tham số TOP-K LUẬT
TOP_K_RULES = 30            # Lấy top 30 luật để làm feature
SORT_BY = "lift"            # Ưu tiên sắp xếp theo lift
DISPLAY_TOP_RULES = 10      # Hiển thị 10 luật tiêu biểu

# Đường dẫn lưu top-K rules
TOP_K_RULES_FP_PATH = "data/processed/top_k_rules_fp.csv"

# Bật/tắt các biểu đồ matplotlib
PLOT_TOP_LIFT = True
PLOT_TOP_CONF = True
PLOT_SCATTER = True
PLOT_NETWORK = True

# Bật/tắt biểu đồ HTML tương tác (Plotly)
PLOT_PLOTLY_SCATTER = True


## Set up

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import time

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

# Biểu đồ tương tác HTML
import plotly.express as px

# Determine correct project root
cwd = os.getcwd()
if os.path.basename(cwd) == "notebooks":
    project_root = os.path.abspath("..")
else:
    project_root = cwd

src_path = os.path.join(project_root, "src")
if src_path not in sys.path:
    sys.path.append(src_path)

from cluster_library import FPGrowthMiner, DataVisualizer  


## Thiết lập style vẽ biểu đồ


In [ ]:
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

visualizer = DataVisualizer()


## Tải basket_bool


In [ ]:
basket_bool = pd.read_parquet(BASKET_BOOL_PATH)

print("=== Thông tin basket_bool ===")
print(f"- Số hoá đơn (rows): {basket_bool.shape[0]:,}")
print(f"- Số sản phẩm (columns): {basket_bool.shape[1]:,}")
print(f"- Tỷ lệ ô = 1 (có mua): {basket_bool.values.mean():.4f}")

basket_bool.head()


## Khai thác tập phổ biến bằng thuật toán FP-Growth


In [ ]:
# Khởi tạo FP-Growth miner
fp_miner = FPGrowthMiner(basket_bool=basket_bool)

start_time = time.time()
frequent_itemsets_fp = fp_miner.mine_frequent_itemsets(
    min_support=MIN_SUPPORT,
    max_len=MAX_LEN,
    use_colnames=True,
)
elapsed_time = time.time() - start_time

print("=== Kết quả khai thác tập mục phổ biến (FP-Growth) ===")
print(f"- Thời gian chạy: {elapsed_time:.2f} giây")
print(f"- Số tập mục phổ biến thu được: {len(frequent_itemsets_fp):,}")

frequent_itemsets_fp.head(10)


In [ ]:
if frequent_itemsets_fp is not None and not frequent_itemsets_fp.empty:
    # Phân phối độ dài các tập mục (1-itemset, 2-itemset, 3-itemset, ...)
    visualizer.plot_itemset_length_distribution(
        frequent_itemsets=frequent_itemsets_fp
    )

    # Top các tập mục phổ biến nhất theo support (ưu tiên itemset có từ 2 sản phẩm trở lên)
    visualizer.plot_top_frequent_itemsets(
        frequent_itemsets=frequent_itemsets_fp,
        top_n=20,
        min_len=2,
    )
else:
    print("Không có frequent itemsets để trực quan hoá.")


## Sinh luật kết hợp từ tập mục phổ biến (FP-Growth)


In [ ]:
rules_fp = fp_miner.generate_rules(
    metric=METRIC,
    min_threshold=MIN_THRESHOLD,
)

# Thêm cột dạng chuỗi dễ đọc
rules_fp = fp_miner.add_readable_rule_str()

print("=== Một vài luật kết hợp đầu tiên (FP-Growth, chưa lọc) ===")
cols_preview = [
    "antecedents_str",
    "consequents_str",
    "support",
    "confidence",
    "lift",
]
rules_fp[cols_preview].head(10)


## Lọc các luật FP-Growth theo ngưỡng support / confidence / lift


In [ ]:
rules_filtered_fp = fp_miner.filter_rules(
    min_support=FILTER_MIN_SUPPORT,
    min_confidence=FILTER_MIN_CONF,
    min_lift=FILTER_MIN_LIFT,
    max_len_antecedents=FILTER_MAX_ANTECEDENTS,
    max_len_consequents=FILTER_MAX_CONSEQUENTS,
)

print("=== Thống kê sau khi lọc luật (FP-Growth) ===")
print(f"- Tổng số luật ban đầu: {rules_fp.shape[0]:,}")
print(f"- Số luật sau khi lọc: {rules_filtered_fp.shape[0]:,}")

rules_filtered_fp[cols_preview].head(10)

# ==================== PHẦN BẮT BUỘC: CHỌN VÀ MINH CHỨNG LUẬT (FP-Growth) ====================

print("\n" + "="*80)
print("BƯỚC CHỌN LUẬT THEO YÊU CẦU MINI PROJECT (FP-Growth)")
print("="*80)

# Tham số Top-K cho feature engineering
TOP_K_RULES = 30            # Lấy top 30 luật để làm feature
SORT_BY = "lift"            # Ưu tiên sắp xếp theo lift
DISPLAY_TOP_RULES = 10      # Hiển thị 10 luật tiêu biểu

# 1. Giải thích lựa chọn tham số
print("\n1. GIẢI THÍCH LỰA CHỌN THAM SỐ (FP-Growth):")
print(f"- min_support = {MIN_SUPPORT}: Đảm bảo luật có đủ độ phổ biến (3% transactions)")
print(f"- min_confidence = {FILTER_MIN_CONF}: Luật đủ tin cậy (40% xác suất)")
print(f"- min_lift = {FILTER_MIN_LIFT}: Chỉ giữ luật có quan hệ thực sự (lift > 1.2)")
print(f"- Ưu tiên sắp xếp theo {SORT_BY.upper()}: Đo lường sức mạnh thực của mối quan hệ")
print(f"- Lấy Top-K = {TOP_K_RULES} luật: Đủ để tạo feature cho clustering, không quá nhiều gây nhiễu")
print(f"- Hiển thị {DISPLAY_TOP_RULES} luật tiêu biểu: Để minh chứng chất lượng luật đầu vào")

# 2. Lấy top-K luật theo tiêu chí
print(f"\n2. LẤY TOP-{TOP_K_RULES} LUẬT THEO {SORT_BY.upper()} (FP-Growth):")

if SORT_BY == "lift":
    top_k_rules_fp = rules_filtered_fp.sort_values(by="lift", ascending=False).head(TOP_K_RULES)
elif SORT_BY == "confidence":
    top_k_rules_fp = rules_filtered_fp.sort_values(by="confidence", ascending=False).head(TOP_K_RULES)
elif SORT_BY == "support":
    top_k_rules_fp = rules_filtered_fp.sort_values(by="support", ascending=False).head(TOP_K_RULES)
else:
    top_k_rules_fp = rules_filtered_fp.head(TOP_K_RULES)

print(f"- Số luật đã chọn: {len(top_k_rules_fp)}")
print(f"- Lift trung bình: {top_k_rules_fp['lift'].mean():.2f}")
print(f"- Confidence trung bình: {top_k_rules_fp['confidence'].mean():.2f}")
print(f"- Support trung bình: {top_k_rules_fp['support'].mean():.4f}")

# 3. Trích xuất 10 luật tiêu biểu để minh chứng (QUAN TRỌNG!)
print(f"\n3. {DISPLAY_TOP_RULES} LUẬT TIÊU BIỂU ĐỂ MINH CHỨNG (FP-Growth, top theo {SORT_BY}):")
print("-" * 100)

# Lấy top DISPLAY_TOP_RULES để hiển thị
display_rules_fp = top_k_rules_fp.head(DISPLAY_TOP_RULES).copy()

# Tạo DataFrame trình bày đẹp
display_df_fp = pd.DataFrame({
    'Antecedents (Nếu mua)': display_rules_fp['antecedents_str'],
    'Consequents (Thì mua)': display_rules_fp['consequents_str'],
    'Support': display_rules_fp['support'].round(4),
    'Confidence': display_rules_fp['confidence'].round(3),
    'Lift': display_rules_fp['lift'].round(2)
})

print(display_df_fp.to_string(index=False))
print("-" * 100)

# 4. Thống kê về các luật đã chọn
print("\n4. THỐNG KÊ LUẬT ĐÃ CHỌN (FP-Growth):")
print(f"- Số luật có lift > 2.0: {(top_k_rules_fp['lift'] > 2.0).sum()}")
print(f"- Số luật có confidence > 0.5: {(top_k_rules_fp['confidence'] > 0.5).sum()}")
print(f"- Số luật có support > 0.03: {(top_k_rules_fp['support'] > 0.03).sum()}")
print(f"- Độ dài antecedent trung bình: {top_k_rules_fp['antecedents'].apply(len).mean():.1f}")
print(f"- Độ dài consequent trung bình: {top_k_rules_fp['consequents'].apply(len).mean():.1f}")

# 5. Lưu top-K rules để dùng cho bước feature engineering
TOP_K_RULES_FP_PATH = "data/processed/top_k_rules_fp.csv"
top_k_rules_fp.to_csv(TOP_K_RULES_FP_PATH, index=False)
print(f"\n5. ĐÃ LƯU TOP-{TOP_K_RULES} LUẬT (FP-Growth):")
print(f"- File: {TOP_K_RULES_FP_PATH}")
print(f"- Sẵn sàng cho bước Feature Engineering")

# ==================== THÊM BIỂU ĐỒ ĐÁNH GIÁ CHẤT LƯỢNG LUẬT ====================

print("\n6. BIỂU ĐỒ ĐÁNH GIÁ CHẤT LƯỢNG LUẬT ĐÃ CHỌN (FP-Growth):")

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Biểu đồ 1: Phân phối lift
axes[0].hist(top_k_rules_fp['lift'], bins=15, edgecolor='black', alpha=0.7, color='skyblue')
axes[0].axvline(x=1.0, color='red', linestyle='--', linewidth=2, label='Lift = 1 (ngẫu nhiên)')
axes[0].set_xlabel('Lift')
axes[0].set_ylabel('Số luật')
axes[0].set_title(f'Phân phối Lift của {TOP_K_RULES} luật (FP-Growth)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Biểu đồ 2: Phân phối confidence
axes[1].hist(top_k_rules_fp['confidence'], bins=15, edgecolor='black', alpha=0.7, color='lightgreen')
axes[1].axvline(x=0.5, color='darkgreen', linestyle='--', linewidth=2, label='Confidence = 0.5')
axes[1].set_xlabel('Confidence')
axes[1].set_ylabel('Số luật')
axes[1].set_title(f'Phân phối Confidence (FP-Growth)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Biểu đồ 3: Lift vs Confidence
scatter = axes[2].scatter(top_k_rules_fp['confidence'], top_k_rules_fp['lift'], 
                         c=top_k_rules_fp['support'], cmap='viridis', 
                         alpha=0.7, s=80, edgecolors='black', linewidth=0.5)
axes[2].set_xlabel('Confidence')
axes[2].set_ylabel('Lift')
axes[2].set_title('Lift vs Confidence (màu = Support) (FP-Growth)')
plt.colorbar(scatter, ax=axes[2], label='Support')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# ==================== PHÂN TÍCH SẢN PHẨM PHỔ BIẾN ====================

print("\n7. PHÂN TÍCH SẢN PHẨM PHỔ BIẾN TRONG LUẬT (FP-Growth):")

# Đếm tần suất sản phẩm xuất hiện
all_products_fp = []
for itemset in top_k_rules_fp['antecedents']:
    all_products_fp.extend(list(itemset))
for itemset in top_k_rules_fp['consequents']:
    all_products_fp.extend(list(itemset))

product_counts_fp = pd.Series(all_products_fp).value_counts().head(10)

print("Top 10 sản phẩm xuất hiện nhiều nhất trong luật (FP-Growth):")
for i, (product, count) in enumerate(product_counts_fp.items(), 1):
    # Cắt bớt tên sản phẩm nếu quá dài
    product_name = product[:50] + "..." if len(product) > 50 else product
    print(f"  {i:2d}. {product_name:53s} - {count:3d} lần")



In [ ]:
# Cell 16: Top theo lift
if PLOT_TOP_LIFT and not rules_filtered_fp.empty:
    visualizer.plot_top_rules_lift(
        rules_df=rules_filtered_fp,
        top_n=TOP_N_RULES,
    )
else:
    if rules_filtered_fp.empty:
        print("Không có luật nào sau khi lọc để vẽ top lift.")
    else:
        print("PLOT_TOP_LIFT = False, bỏ qua biểu đồ top lift.")


In [ ]:
# Cell 17: Top theo confidence
if PLOT_TOP_CONF and not rules_filtered_fp.empty:
    visualizer.plot_top_rules_confidence(
        rules_df=rules_filtered_fp,
        top_n=TOP_N_RULES,
    )
else:
    if rules_filtered_fp.empty:
        print("Không có luật nào sau khi lọc để vẽ top confidence.")
    else:
        print("PLOT_TOP_CONF = False, bỏ qua biểu đồ top confidence.")


In [ ]:
# Cell 18: Scatter support–confidence
if PLOT_SCATTER and not rules_filtered_fp.empty:
    visualizer.plot_rules_support_confidence_scatter(
        rules_df=rules_filtered_fp,
    )
else:
    if rules_filtered_fp.empty:
        print("Không có luật nào sau khi lọc để vẽ scatter.")
    else:
        print("PLOT_SCATTER = False, bỏ qua biểu đồ scatter.")


In [ ]:
# Cell 19: Scatter Plotly
if PLOT_PLOTLY_SCATTER and not rules_filtered_fp.empty:
    visualizer.plot_rules_support_confidence_scatter_interactive(
        rules_df=rules_filtered_fp,
    )
else:
    if rules_filtered_fp.empty:
        print("Không có luật nào sau khi lọc để vẽ scatter Plotly.")
    else:
        print("PLOT_PLOTLY_SCATTER = False, bỏ qua biểu đồ Plotly.")


In [ ]:
# Cell 20: Network graph
if PLOT_NETWORK and not rules_filtered_fp.empty:
    visualizer.plot_rules_network(
        rules_df=rules_filtered_fp,
        max_rules=min(TOP_N_RULES, 30),
    )
else:
    if rules_filtered_fp.empty:
        print("Không có luật nào sau khi lọc để vẽ network graph.")
    else:
        print("PLOT_NETWORK = False, bỏ qua network graph.")


In [ ]:
fp_miner.save_rules(
    output_path=RULES_OUTPUT_PATH,
    rules_df=rules_filtered_fp,
)

print("Đã lưu luật FP-Growth đã lọc:")
print(f"- File: {RULES_OUTPUT_PATH}")
print(f"- Số luật: {rules_filtered_fp.shape[0]:,}")
